In [ ]:
# %% [Cell 1] MIP selection in Jupyter - four CSVs in the same folder
# SPDX-License-Identifier: MIT
# Copyright (c) 2025 Hamid
#
# This cell:
# - Finds or uses four CSV files with columns [time_ns, chan, ph]
# - Builds ADC histograms per channel and selects a MIP window by fractional-height crossings
# - Saves plots and a summary CSV to ./mip_outputs
# - Optionally writes LICENSE, README.md, and CITATION.cff for publishing

import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches
from IPython.display import display
from typing import List, Tuple

# Savitzky-Golay smoothing
try:
    from scipy.signal import savgol_filter
    _HAVE_SCIPY = True
except Exception:
    _HAVE_SCIPY = False

# ---------------- User knobs ----------------
# If you want to specify files explicitly, list them here. Leave empty to auto-pick four CSVs in the folder.
FILES: List[str] = []  # example: ["file1.csv","file2.csv","file3.csv","file4.csv"]

CHANNELS = [1, 2, 3]
OUT_DIR = Path("./mip_outputs").resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Peak search band and MIP window settings
MUON_RANGE: Tuple[float, float] = (170.0, 360.0)
MIP_FRAC = 0.50          # fractional height relative to a local baseline
SG_WIN = 21              # odd window for smoothing
SG_POLY = 3
GUARD_BINS = 10          # do not attach edges too close to the peak
MIN_WIDTH = 40           # minimum width in bins
SAFE_MARG = 4            # small expansion each side
PH_EDGES = np.arange(0, 1025, 1)
BIN_CENTERS = 0.5 * (PH_EDGES[:-1] + PH_EDGES[1:])

# Plot styling
plt.rcParams.update({"font.family":"Times New Roman", "figure.dpi":140})
TEXT_CFG = {"title": 18, "label": 18, "tick": 16, "legend": 16}
ROLE_COLORS = ["#1f77b4", "#d62728", "#2ca02c", "#ff7f0e"]

def apply_text_sizes(ax, cfg=TEXT_CFG, legend_obj=None):
    ax.title.set_fontsize(cfg["title"])
    ax.xaxis.label.set_size(cfg["label"])
    ax.yaxis.label.set_size(cfg["label"])
    ax.tick_params(axis="both", labelsize=cfg["tick"])
    if legend_obj is not None:
        for txt in legend_obj.get_texts():
            txt.set_fontsize(cfg["legend"])

# --------------- Helpers ---------------
def list_four_csvs_here() -> List[Path]:
    csvs = sorted([p for p in Path(".").glob("*.csv")])
    return csvs[:4]

def load_threecol_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, header=None, names=["time_ns","chan","ph"])
    df["time_ns"] = pd.to_numeric(df["time_ns"], errors="coerce")
    df["chan"]    = pd.to_numeric(df["chan"],    errors="coerce")
    df["ph"]      = pd.to_numeric(df["ph"],      errors="coerce")
    df = df.dropna(subset=["time_ns","chan","ph"]).copy()
    df["chan"] = df["chan"].astype(int)
    return df

def hist_channel(ph: np.ndarray, edges: np.ndarray) -> np.ndarray:
    c, _ = np.histogram(ph, bins=edges)
    return c.astype(float)

def sg_smooth(y: np.ndarray, win=SG_WIN, poly=SG_POLY) -> np.ndarray:
    if not _HAVE_SCIPY:
        # Fallback to simple moving average if SciPy is not available
        w = max(5, int(win) | 1)
        if y.size < w:
            return y.copy()
        kernel = np.ones(w, dtype=float) / w
        return np.convolve(y, kernel, mode="same")
    n = len(y)
    w = int(win) | 1
    if n < 5:
        return y.copy()
    if w > n-1:
        w = max(5, ((n-1)//2)*2 + 1)
    if w < 5 or w >= n:
        return y.copy()
    return savgol_filter(y, window_length=w, polyorder=min(poly, w-1), mode="interp")

def estimate_bg_local(y: np.ndarray) -> float:
    if y.size < 20:
        return float(np.median(y))
    q = np.quantile(y, 0.2)
    return float(np.median(y[y <= q]))

def pick_mip_peak(counts: np.ndarray, x: np.ndarray, muon_range: Tuple[float,float]) -> int:
    sm = sg_smooth(counts, SG_WIN, SG_POLY)
    mask = (x >= muon_range[0]) & (x <= muon_range[1])
    if not np.any(mask):
        return int(np.argmax(sm))
    roi = sm.copy()
    roi[~mask] = -np.inf
    i0 = int(np.nanargmax(roi))
    L = max(0, i0-25); R = min(len(counts)-1, i0+25)
    i_ref = int(np.argmax(counts[L:R+1])) + L
    return i_ref

def mip_window_from_fraction(counts: np.ndarray, x: np.ndarray, i0: int,
                             frac=MIP_FRAC, guard=GUARD_BINS,
                             min_width=MIN_WIDTH, safe=SAFE_MARG,
                             muon_range: Tuple[float,float]=MUON_RANGE) -> Tuple[int,int,np.ndarray]:
    sm = sg_smooth(counts, SG_WIN, SG_POLY)
    n  = len(sm)

    # Wide neighborhood around peak, clamped by a generous muon envelope
    l_nei = int(np.searchsorted(x, max(muon_range[0]-20, x[i0]-140), side="left"))
    r_nei = int(np.searchsorted(x, min(muon_range[1]+40, x[i0]+200), side="right")) - 1
    l_nei = max(0, l_nei); r_nei = min(n-1, r_nei)

    b  = estimate_bg_local(sm[l_nei:r_nei+1])
    pk = float(sm[i0])
    thr = b + float(frac) * max(pk - b, 1.0)

    # Left crossing
    left_lo = l_nei
    left_hi = max(left_lo, i0 - guard)
    segL = sm[left_lo:i0+1]
    idxL = np.where(segL <= thr)[0]
    iL = int(left_lo + idxL[-1]) if idxL.size else max(left_lo, i0 - 3*SG_WIN)

    # Right crossing
    right_lo = min(i0 + guard, r_nei)
    right_hi = r_nei
    segR = sm[i0:right_hi+1]
    idxR = np.where(segR <= thr)[0]
    iR = int(i0 + idxR[0]) if idxR.size else min(r_nei, i0 + 3*SG_WIN)

    # Safety expansion
    iL = max(0, iL - safe)
    iR = min(n-1, iR + safe)

    # Enforce minimum width
    if iR - iL < min_width:
        deficit = min_width - (iR - iL)
        shiftL = deficit // 2
        shiftR = deficit - shiftL
        iL = max(0, iL - shiftL)
        iR = min(n-1, iR + shiftR)

    # Clamp to a soft muon envelope
    floor_adc = max(muon_range[0] - 10, x[i0] - 200)
    ceil_adc  = min(muon_range[1] + 40, x[i0] + 240)
    i_floor   = int(np.searchsorted(x, floor_adc, side="left"))
    i_ceil    = int(np.searchsorted(x, ceil_adc, side="right")) - 1

    iL = max(iL, i_floor)
    iR = min(iR, i_ceil)
    iL = int(np.clip(iL, 0, n-2))
    iR = int(np.clip(max(iR, iL+1), 1, n-1))

    return iL, iR, sm

def plot_linear_mip(title: str, x: np.ndarray, counts: np.ndarray, sm: np.ndarray,
                    iL: int, iR: int, color: str, outfile: Path, label: str):
    plt.figure(figsize=(9.6,5.6))
    plt.plot(x, counts, color=color, lw=1.8, label=label)
    plt.plot(x, sm,     color="black", lw=2.0, label="Smoothed")

    x0, x1 = float(x[iL]), float(x[iR])
    plt.axvspan(x0, x1, color=(0.63,0.13,0.94,0.22), lw=0, label=f"MIP [{int(x0)}, {int(x1)}]")
    plt.axvline(x0, color=(0.63,0.13,0.94,0.9), lw=1.0)
    plt.axvline(x1, color=(0.63,0.13,0.94,0.9), lw=1.0)

    handles, labels = plt.gca().get_legend_handles_labels()
    leg = plt.legend(handles, labels, frameon=True)

    ymax = max(np.nanmax(counts), np.nanmax(sm)) if np.isfinite(np.nanmax(counts)) else 1.0
    plt.ylim(0, ymax*1.12)
    plt.xlim(0, 1024)
    plt.xlabel("Pulse Height (ADC)")
    plt.ylabel("Counts")
    plt.title(title + " (Linear)")
    plt.grid(True, alpha=0.3)

    apply_text_sizes(plt.gca(), TEXT_CFG, legend_obj=leg)
    plt.tight_layout()
    plt.savefig(outfile, bbox_inches="tight")
    plt.close()

# --------------- Main in-notebook run ---------------
# Resolve four files: explicit list or auto-pick
if FILES:
    file_paths = [Path(f).resolve() for f in FILES]
else:
    picks = list_four_csvs_here()
    if len(picks) < 4:
        raise RuntimeError("Fewer than four CSV files found in the current folder. Add files or fill FILES list.")
    file_paths = [p.resolve() for p in picks]

print("Using files:")
for p in file_paths:
    print(" -", p.name)

summary_rows = []
for idx, fpath in enumerate(file_paths, start=1):
    if not fpath.exists():
        print(f"[WARN] Missing file: {fpath}")
        continue

    df = load_threecol_csv(fpath)
    for ch in CHANNELS:
        ph = df.loc[df["chan"]==ch, "ph"].to_numpy()
        if ph.size == 0:
            summary_rows.append(dict(
                file=fpath.name, label=fpath.stem, channel=ch,
                peak_adc=np.nan, mip_min=np.nan, mip_max=np.nan,
                mip_count=np.nan, total_count=0
            ))
            continue

        counts = hist_channel(ph, PH_EDGES)
        i0 = pick_mip_peak(counts, BIN_CENTERS, MUON_RANGE)
        iL, iR, sm = mip_window_from_fraction(
            counts, BIN_CENTERS, i0,
            frac=MIP_FRAC, min_width=MIN_WIDTH, muon_range=MUON_RANGE
        )

        peak_adc = int(BIN_CENTERS[i0])
        mip_min  = int(BIN_CENTERS[iL])
        mip_max  = int(BIN_CENTERS[iR])
        mip_count = int(np.sum(counts[iL:iR+1]))
        total_count = int(np.sum(counts))

        summary_rows.append(dict(
            file=fpath.name, label=fpath.stem, channel=ch,
            peak_adc=peak_adc, mip_min=mip_min, mip_max=mip_max,
            mip_count=mip_count, total_count=total_count
        ))

        png_name = f"{fpath.stem}__Ch{ch}__MIP_lin.png"
        plot_linear_mip(
            title=f"MIP - {fpath.stem} - Channel {ch}",
            x=BIN_CENTERS, counts=counts, sm=sm,
            iL=iL, iR=iR, color=ROLE_COLORS[(idx-1) % len(ROLE_COLORS)],
            outfile=OUT_DIR / png_name, label=fpath.stem
        )

# Save and show summary
df_sum = pd.DataFrame(summary_rows)
csv_path = OUT_DIR / "mip_summary.csv"
df_sum.to_csv(csv_path, index=False)
print(f"\nSaved {len(df_sum)} rows to {csv_path}")
display(df_sum.head(12))

# --------------- Optional: write repo files for GitHub ---------------
WRITE_REPO_FILES = True  # set False if you do not want these created
if WRITE_REPO_FILES:
    # LICENSE
    (Path("LICENSE")).write_text(
        "MIT License\n\n"
        "Copyright (c) 2025 Hamid\n\n"
        "Permission is hereby granted, free of charge, to any person obtaining a copy\n"
        "of this software and associated documentation files to deal in the Software\n"
        "without restriction, including without limitation the rights to use, copy,\n"
        "modify, merge, publish, distribute, sublicense, and to permit persons to whom\n"
        "the Software is furnished to do so, subject to the following conditions:\n\n"
        "The above copyright notice and this permission notice shall be included in all\n"
        "copies or substantial portions of the Software.\n\n"
        "THE SOFTWARE IS PROVIDED AS IS, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\n"
        "IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,\n"
        "FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE\n"
        "AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER\n"
        "LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,\n"
        "OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE\n"
        "SOFTWARE.\n",
        encoding="utf-8"
    )

    # README.md
    (Path("README.md")).write_text(
        "# MIP selection\n\n"
        "Peak-centered MIP selection from ADC histograms. Input CSVs must have three columns without a header: time_ns, chan, ph.\n\n"
        "## Install\n"
        "```bash\n"
        "python -m venv .venv && . .venv/bin/activate  # Windows: .venv\\Scripts\\activate\n"
        "pip install numpy pandas matplotlib scipy\n"
        "```\n\n"
        "## Use in Jupyter\n"
        "Paste the single cell from the notebook and run. By default it auto-picks four CSV files in the current folder. "
        "To specify files explicitly, fill FILES at the top of the cell.\n\n"
        "Outputs go to ./mip_outputs\n\n"
        "## Method\n"
        "Savitzky-Golay smoothing for the histogram, strict peak pick inside a muon ADC band, then MIP bounds by fractional-height crossings relative to a local baseline, with guard and minimum width.\n\n"
        "## License\n"
        "MIT\n",
        encoding="utf-8"
    )

    # CITATION.cff
    (Path("CITATION.cff")).write_text(
        "cff-version: 1.2.0\n"
        "message: Please cite this software if you use it.\n"
        "title: MIP selection for ADC histograms\n"
        "authors:\n"
        "  - family-names: Hamid\n"
        "    given-names: -\n"
        "license: MIT\n"
        "date-released: 2025-10-14\n",
        encoding="utf-8"
    )

    print("Wrote LICENSE, README.md, and CITATION.cff in the current folder.")


usage: ipykernel_launcher.py [-h] [-o OUT] [-c CHANNELS [CHANNELS ...]]
                             [--mip-frac MIP_FRAC] [--muon-min MUON_MIN]
                             [--muon-max MUON_MAX] [--min-width MIN_WIDTH]
                             file1 file2 file3 file4
ipykernel_launcher.py: error: the following arguments are required: file1, file2, file3, file4


SystemExit: 2

c:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
